task name: Implement LightGBM Regression
task description: Implement the provided C# code for training and evaluating a LightGBM regression model.

In [ ]:
#r "nuget:Microsoft.ML.LightGbm"

In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;

In [ ]:
var mlContext = new MLContext(seed: 0);

In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;

private static IEnumerable<DataPoint> GenerateRandomDataPoints(int count, int seed = 0)
{
    var random = new Random(seed);
    for (int i = 0; i < count; i++)
    {
        float label = (float)random.NextDouble();
        yield return new DataPoint
        {
            Label = label,
            Features = Enumerable.Repeat(label, 50).Select(x => x + (float)random.NextDouble()).ToArray()
        };
    }
}

public class DataPoint
{
    public float Label { get; set; }
    [VectorType(50)]
    public float[] Features { get; set; }
}

var dataPoints = GenerateRandomDataPoints(1000).ToList();
var trainingData = mlContext.Data.LoadFromEnumerable(dataPoints);

In [ ]:
var pipeline = mlContext.Regression.Trainers.LightGbm(labelColumnName: nameof(DataPoint.Label), featureColumnName: nameof(DataPoint.Features));
var model = pipeline.Fit(trainingData);

In [ ]:
var testData = mlContext.Data.LoadFromEnumerable(GenerateRandomDataPoints(5, seed: 123));
var transformedTestData = model.Transform(testData);

In [ ]:
public class Prediction
{
    public float Label { get; set; }
    public float Score { get; set; }
}

var predictions = mlContext.Data.CreateEnumerable<Prediction>(transformedTestData, reuseRowObject: false).ToList();
foreach (var p in predictions)
    Console.WriteLine($"Label: {p.Label:F3}, Prediction: {p.Score:F3}");

In [ ]:
var metrics = mlContext.Regression.Evaluate(transformedTestData);

Console.WriteLine("Mean Absolute Error: " + metrics.MeanAbsoluteError);
Console.WriteLine("Mean Squared Error: " + metrics.MeanSquaredError);
Console.WriteLine("Root Mean Squared Error: " + metrics.RootMeanSquaredError);
Console.WriteLine("RSquared: " + metrics.RSquared);